## LU Factorization (LU Decomposition)

Any non-singular matrix $\mathbf{A}$ can be factored into a lower triangular matrix $\mathbf{L}$, and upper triangular matrix $\mathbf{U}$ using procedures we have already established with Gaussian elimination. This proves very useful for numerical computation and is, in fact, one of the most common ways most packaged linear algebra solvers solve non-sparse linear systems.

Previously, we learned that by using Gaussian elimination we can solve the linear system $\mathbf{A}\vec{x} = \vec{b}$ in $O(\frac{1}{3}n^3)$ arithmetic operations to determine $\vec{x}$. It turns out if $\mathbf{A}$ has the form $\mathbf{A=LU}$ we can solve for $\vec{x}$ using a two step process. First we let $\vec{y}=\mathbf{U}\vec{x}$ and solve the system for $L\vec{y}=\vec{b}$ for $\vec{y}$. Since $\mathbf{L}$ is lower triangular we use a forward substitution process that only takes $O(n^2)$ operations. Once $\vec{y}$ is known, the upper triangular system $\mathbf{U}\vec{x}=\vec{y}$ can be solved with back substitution in $O(n^2)$ operations. Therefore using this procedure we can reduce the solution of $
\mathbf{A}\vec{x} = \vec{b}$ from $O(\frac{1}{3}n^3)$ to $O(2n^2)$ operations. For large systems this can reduce the calculation time by more than 95%. Determining the $\mathbf{L}$ and $\mathbf{U}$ matrices still takes $O(\frac{1}{3}n^3)$, but it only has to be done for a single $\mathbf{A}$ matrix and can be used efficiently to solve for $\vec{x}$ with many right-hand side vectors, $\vec{b}$.

We already know one way to transform $\mathbf{A}$ into $\mathbf{U}$ by using Gaussian elimination. We will leave the proof for linear algebra class, but we find $\mathbf{L}$ by performing negations of the same operations on the identity matrix in reverse order. For example when going $\mathbf{A}\rightarrow \mathbf{U}$ if we perform the row operation

$$
E_i - m_{ji}E_j\rightarrow E_i
$$

to undo this and return to $\mathbf{A}$ we would perform

$$
E_i+m_{ji}E_j\rightarrow E_i
$$

by performing the second operation on the identity matrix with the same dimensions as $\mathbf{A}$ we will end up with an $\mathbf{L}$. I emphasize *an* here because this is only one way to decompose $\mathbf{A}$ into $\mathbf{L}$ and $\mathbf{U}$, there are other methods and an infinite number of $\mathbf{L}$ and $\mathbf{U}$ matrices, i.e. they are not unique.

#### LU Example

We will look at a simple example and write out the row operations.

$$
\mathbf{A} = \begin{bmatrix}1&1&0\\2&1&-1\\3&-1&-1\end{bmatrix}
\xrightarrow{E_2-2E_1}
\begin{bmatrix}1&1&0\\0&-1&-1\\3&-1&-1\end{bmatrix}
\xrightarrow{E_3-3E_1}
\begin{bmatrix}1&1&0\\0&-1&-1\\0&-4&-1\end{bmatrix}
\xrightarrow{E_3-4E_2}
\begin{bmatrix}1&1&0\\0&-1&-1\\0&0&3\end{bmatrix} = \mathbf{U}
$$

$$
\mathbf{I} = \begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}
\xrightarrow{E_3+4E_2}
\begin{bmatrix}1&0&0\\0&1&0\\0&4&1\end{bmatrix}
\xrightarrow{E_3+3E_1}
\begin{bmatrix}1&0&0\\0&1&0\\3&4&1\end{bmatrix}
\xrightarrow{E_2+2E_1}
\begin{bmatrix}1&0&0\\2&1&0\\3&4&1\end{bmatrix} = \mathbf{L}
$$

Let's check our results with Python.

In [2]:
import numpy as np

In [3]:
L = np.array([[1,0,0],[2,1,0],[3,4,1]])
U = np.array([[1,1,0],[0,-1,-1],[0,0,3]])
L @ U

array([[ 1,  1,  0],
       [ 2,  1, -1],
       [ 3, -1, -1]])

Notice that the entries in $\mathbf{L}$ below the diagonal are simply the $m_{ij}$'s therefore we do not actually have to perform the row operations we can simply insert the $m_{ij}$ components into their proper place and move forward.

### Psuedocode for a simple LU factorization

Consider the matricies:

\begin{equation}
\mathbf{A} = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
\mathbf{L} = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
\mathbf{U} = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
\end{equation}

  1. Initialize $L$ to an identity matrix, $I$, of dimension $n$ by $n$ and $U = A$.
  1. For $i = 1$, ..., $n$ do Step A.
    1. For $j=i+1, ..., n$ do Steps a-b
      1. Set $I_{ji}=u_{ji}/u_{ii}$
      1. Perform $U_j = (U_j-I_{ji}U_i)$ (where $U_i, U_j$ represent the $i$ and $j$ rows of the matrix $U$, respectively)

### Psuedocode for a simple PLU factorization

If we observe the pseudocode presented previously we can see that something as simple as $a_{ii}=0$ will cause our algorithm to fail.  We can prevent this from happening by doing a row exchange with a row that has a nonzero value in the $i^\mathrm{th}$ column.  Like Gauss elimination there are several pivot strategies that can be utilized.  For simplicity, we will exchange any row that has a zero on the diagonal with the first row below it that has a nonzero number in that column.  Here we have to keep track of the row exchanges by creating a permutation matrix, $\mathbf{P}$ by performing the same row exchanges to an identity matrix.  This gives a matrix with precisely one nonzero entry in each row and in each column and each nonzero entry is 1.  We will need $\mathbf{P}$ later, when we use the LU factorization to solve the permutated system of equations, $\mathbf{P A}\vec{x} = \mathbf{P} \vec{b}$.

Consider the matrices:

\begin{equation}
\mathbf{A} = \begin{bmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{bmatrix},
\mathbf{L} = \begin{bmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{bmatrix},
\mathbf{U} = \begin{bmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{bmatrix},
\mathbf{P} = \begin{bmatrix}p_{11}&p_{12}&...&p_{1n}\\p_{21}&\ddots& &p_{2n}\\\vdots& &\ddots& \vdots\\p_{n1}&...&...&p_{nn}\end{bmatrix}
\end{equation}

1. Initialize $\mathbf{L = P = I}$ of dimension n by n and $\mathbf{U = A}.
2. For i = 1, ..., n do Step 3-7.

    3. Let k = i, 
4. While Subscript[a, ii]=0, do Steps 5-7
5. Swap row Subscript[A, i] with row Subscript[A, k+1]
6. Swap row Subscript[P, i] with row Subscript[P, k+1]
7. Increment k by 1.
8. For j = i+1, ..., n do Steps 4-5
4. Set Subscript[l, ji]= Subscript[u, ji]/Subscript[u, ii]
5. Perform Subscript[U, j]=(Subscript[U, j]- Subscript[l, ji] Subscript[U, i])					(where Subscript[U, i], Subscript[U, j] represent the i and j rows of the matrix U, repectivley) 

In [15]:
def lu(A):
    
    #Get the number of rows
    N = A.shape[0]
    
    U = A
    L = np.eye(N, dtype=np.double)
    
    #Loop over rows
    for i in range(N):
            
        #Eliminate entries below i with row operations on U and
        #reverse the row operations to manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U

## Psuedocode for solving equations after LU factorization

Once we have $L$ and $U$ we can solve for as many right-hand side vectors $\bar{b}$ as desired very quickly using the following two step process. First we let $\bar{y}=U\bar{x}$ and then solve for $L\bar{y}=\bar{b}$ for $\bar{y}$ by using forward substitution. The pseudocode for this is as follows:

  1. Set $y_1 = b_1/I_{11}$
  1. For $i=2, 3, ..., n$ do Step 3.
  1. $y_i = (b_i - \sum\nolimits_{j=1}^{i-1} I_{ij}y_j)/I_{ii}$


In [30]:
def forward_substitution(L, b):
    
    N = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  Initializing 
    #with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom up), starting with the second to
    #last row, because the last row solve was completed in the last step.
    for i in range(1, N):
        y[i] = (b[i] - np.dot(L[i,i:], y[i:])) / L[i,i]
        
    return y

Then solve $U\bar{x} = \bar{y}$ for $\bar{x}$ using backward substitution. The psuedocode for this is as follows:

  1. Set $x_n = y_n / u_{nn}$
  1. For $i = n-1, n-2, ..., 1$ do Step 3.
  1. $x_i = (y_i - \sum\nolimits_{j=i+1}^{n} u_{ij}x_j)/u_{ii}$
 

For a $\mathbf{PLU}$ factorization would have the additional step of permutating the right-hand side vector $\bar{b} = P\bar{b}$ before doing the forward substitution to solve for $\bar{y}$.

In [31]:
def back_substitution(U, y):
    
    N = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  Initializing 
    #with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), starting with the second to
    #last row, because the last row solve was completed in the last step.
    for i in range(N-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

In [32]:
def lu_solve(A, b):
    
    L, U = lu(A)
    
    y = forward_substitution(L, b)
    
    return back_substitution(U, y)

In [34]:
A = np.array([[1, 4, 5], [6, 8, 22], [32, 5., 5]])
b = np.array([1, 2, 3.])

lu_solve(A, b)

array([ 1.59625668, -0.10895722, -0.03208556])

In [35]:
np.linalg.solve(A, b)

array([ 1.59625668, -0.10895722, -0.03208556])

## LU factorization, another look.

Let's condsier a generic $LU$ factorization, for simplicity we will consider a set of 3 x 3 matrices, but these ideas will apply in the $n$x$n$ case.

$$
LU = \begin{pmatrix}I_{11}&0&0\\I_{21}&I_{22}&0\\I_{31}&I_{32}&I_{33}\end{pmatrix}
\begin{pmatrix}u_{11}&u_{12}&u_{13}\\0&u_{22}&u_{23}\\0&0&u_{33}\end{pmatrix} = 
\begin{pmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{pmatrix} = A
$$

If we multiply the two left side matrices together, we have

$$
\begin{pmatrix}I_{11}u_{11}&I_{11}u_{12}&I_{11}u_{12}\\I_{21}u_{11}&I_{21}u_{12}+I_{22}u_{22}&I_{21}u_{13}+I_{22}u_{23}\\I_{31}u_{11}&I_{31}u_{12}+I_{32}u_{22}&I_{31}u_{13}+I_{32}u_{23}+I_{33}u_{33}\end{pmatrix} = 
\begin{pmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{pmatrix}
$$

Now if we take the same approah as earlier where the $I_{ii}$'s are the 1's we can solve the first row of equations trivially, namely:

$u_{11} = a_{11}, u_{12} = a_{12}, u_{13} = a_{13},$ then we have enough information to solve the rest of the first column,

$I_{21} = a_{21}/a_{11}, I_{31}=a_{31}/a_{11},$ and the rest of the second row,

$u_{22} = (a_{22}-a_{21}^{2}/a_{11}), u_{23} = (a_{23}-a_{21}a_{23}/a_{11}),$ etc.

When this procedure is generalized it is known as the Doolittle alogrithm. There is a similar procedure known as Crout's method that uses the 1's on the diagonal of the $U$ matrix. The generalization of these methods will be shown in the upcoming pages describing their pseudocode.


## Pseudocode for Doolittle algorithm. 

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
U = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
$$

  1. For $k = 1, 2, ..., n$ do Steps 2-6
  1. Set $I_{kk}=1$
  1. For $j=k, k+1, ..., n$ do Step 4.
  1. $u_{kj} = a_{kj} - \sum\nolimits_{m=1}^{k-1}I_{km}u_{mj}$
  1. For $i=k+1, k+2, ..., n$ do Step 6.
  1. $I_{ik}=\frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}u_{mk}}{u_{kk}}$

In [ ]:
def doolittle(A):
    
    N = A.shape[0]
    
    L = np.zeros((N, N), dtype=np.double)
    U = np.zeros((N, N), dtype=np.double)
    
    for k in range(N):
        
        L[k, k] = 1.

## Pseudocode for Crout algorithm.

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
U = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
$$

  1. For $k = 1, 2, ..., n$ do Steps 2-6
  1. Set $I_{kk} = a_{kk}-\sum\nolimits_{m=1}^{k-1}I_{km}u_{mk}$
  1. For $j = k, k+1, ..., n$ do Step 4.
  1. $u_{kj} = \frac{a_{kj}-\sum\nolimits_{m=1}^{k-1}I_{km}u_{mj}}{I_{kk}}$
  1. For $i = k+1, k+2, ..., n$ do Step 6.
  1. $I_{ik} = \frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}u_{mk}}{u_{kk}}$

In [ ]:
def crout(A, b):
    
    N = A.shape[0]
    
    L, U = lu(A)
    
    for k in range(N):
        
        L[k, k] = A[k, k] - 

## Pseudocode for Cholesky decomposition.

If matrix $A$ is symmetric and positive definite, then there exists a lower triangular matrix $L$ such that $A=LL^T$. This is just a special case of the $LU$ decomposition, $U=L^T$. The algorithm is slightly simpler than the Doolittle or Crout methods.

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
$$

  1. For $k=1,2,...,n$ do Steps 2-4
  1. Set $I_{kk} = \sqrt{a_{kk}-\sum\nolimits_{m=1}^{k-1}I_{km}^2}$
  1. For $i=k+1,k+2,...,n$ do Step 4.
  1. $I_{ik}=\frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}I_{km}}{I_{kk}}$

##Solving for the inverse of $A$ once the $LU$ decomposition is known.

Once the $LU$ decomposition of $A$ is complete it is straightforward to find the inverse of $A$, using the same forward and backward substituion process we used when solving for an arbitrary right hand side vector $\bar{b}$. Except we will do the procedure $n$ times, where $n$ is the number of columns of $A$ and $\bar{b} = [\bar{b_1};\bar{b_2};...;\bar{b_n}] = I$ are the columns of the Identity matrix. Stated differently, we will use the columns of the identity matrix as individual right-hand side vectors, $\bar{b}$, in order to solve for the inverse column-by-column.

The pseudocode is as follows:

  1. Set $\bar{b} = I$ with dimension $n$x$n$
  1. For $k = 1, 2, ..., n$ do Steps 3-9
  1. Set $y_{1k}=\frac{b_{1k}}{I_{11}}$
  1. For $i = 2, 3, ..., n$ do Step 5.
  1. $y_{ik} = \frac{b_{ik}-\sum\nolimits_{j=1}^{i-1}I_{ij}y_{jk}}{I_{ii}}$
  1. Set $x_{nk} = \frac{y_{nk}}{u_{nn}}$
  1. For $i = n-1, n-2, ..., 1$ do Step 8.
  1. $x_{ik}= \frac{y_{ik}-\sum\nolimits_{j=i+1}^{n}u_{ij}x_{jk}}{u_{ii}}$

##Determinant of a Matrix.

You might reall from linear algebra that there are several ways of computing the determinant of a matrix (e.g. Leibniz formula, Laplace formula, Cramer's rule, etc.), however, none of these are computationally as efficient as using the $LU$ decomposition and a few properties of determinants to solve for the determinate of a matrix $A$. Recall,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For $A = LU, \rightarrow det(A) = det(L)det(U)$

also for an upper (or lower) triangular matrix. The determinate of the matrix is simply the product of the diagonal entries. Therefore, if we solve for $L$ and $U$ using the Doolittle method, where there are 1's on the diagonal of the $L$ matrix, then the determinate of $L$ is 1. Thus,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$det(A) = 1*det(U) = \prod_{j=1}^{n}u_{jj}$

Similarly, for a $PLU$ decomposition,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$det(A) = det(P)det(L)det(U) = det(P)*\prod_{j=1}^{n}u_{jj}$

but, $P$ is just a permutation of the Identity matrix. Let's observe what happens when we do five semi-random row permutations of the identity matrix and calculate the determinate of $P$ each time.

In [7]:
import numpy as np
from numpy import matrix
n = 3
P = np.matrix(np.identity(n))
print P


[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


Therefore all we need to do is keep track of the number of row permutations and the $det(P)$ will have the following properties:

$det(P) = \{ _{ 1\,for\,even\,number\,of\,permutations}^{-1\,for\,odd\,number\,of\,permutations}$